## How-to guide for Account Takeover and Defense use-case on Abacus.AI platform
This notebook provides you with a hands on environment to build an account takeover detection & protection model using the Abacus.AI Python Client Library.

We'll be using the [Insider Threat Test Dataset](https://s3.amazonaws.com/realityengines.exampledatasets/accounttakeover/kdd_500k.csv), which contains information about attacks on certain accounts.

1. Install the Abacus.AI library.

In [ ]:
!pip install abacusai

     |████████████████████████████████| 40 kB 2.5 MB/s 
     |████████████████████████████████| 2.3 MB 8.2 MB/s 
  Created wheel for abacusai: filename=abacusai-0.32.9-py3-none-any.whl size=58147 sha256=eb42912ad51747e61b7cc867f7cbda973085f6223a2c3eede6a011bd13202b7b
  Stored in directory: /root/.cache/pip/wheels/f2/79/18/e8a7e9f30fc534e6f5b2a67eebac0add054bbabfa983663ec5
Successfully built abacusai


We'll also import pandas and pprint tools for neat visualization in this notebook:

In [ ]:
import pandas as pd # A tool we'll use to download and preview CSV files
import pprint # A tool to pretty print dictionary outputs
pp = pprint.PrettyPrinter(indent=2)

2. Add your Abacus.AI [API Key](https://abacus.ai/app/profile/apikey) generated using the API dashboard as follows:

In [ ]:
#@title Abacus.AI API Key

api_key = '2fdecde877dc45fab937eff82b70eff0'  #@param {type: "string"}

3. Import the Abacus.AI library and instantiate a client.

In [ ]:
from abacusai import ApiClient
client = ApiClient(api_key)

## 1. Create a Project

Abacus.AI projects are containers that have datasets and trained models. By specifying a business **Use Case**, Abacus.AI tailors the deep learning algorithms to produce the best performing model catered specifically for your data.

We'll call the `list_use_cases` method to retrieve a list of the Use Cases currently available on the Abacus.AI platform.

In [ ]:
client.list_use_cases()

[UseCase(use_case='UCPLUGANDPLAY',
   pretty_name='Plug & Play Your Tensorflow Model',
   description='Upload your already trained model and leverage our model serving infrastructure.. Host your models on our infrastructure and get a JSON api with auto scaling and more!'),
 UseCase(use_case='EMBEDDINGS_ONLY',
   pretty_name='Vector Matching Engine',
   description='Upload embeddings and leverage our similarity search infrastructure.. Scale to high traffic, update your index in near realtime'),
 UseCase(use_case='MODEL_WITH_EMBEDDINGS',
   pretty_name='Tensorflow Model With Vector Matching Engine',
   description='Upload your already trained model and leverage our model serving infrastructure.. Host your models on our infrastructure and get a JSON api with auto scaling and more!'),
 UseCase(use_case='TORCH_MODEL_WITH_EMBEDDINGS',
   pretty_name='PyTorch Model With Vector Matching Engine',
   description='Upload your already trained model and leverage our model serving infrastructure.. H

In this notebook, we're going to create an account takeover prediction model using the Insider Threat Test dataset. The 'FRAUD_ACCOUNT' use case is best tailored for this situation.

In [ ]:
#@title Abacus.AI Use Case

use_case = 'FRAUD_ACCOUNT'  #@param {type: "string"}

By calling the `describe_use_case_requirements` method we can view what datasets are required for this use_case.

In [ ]:
for requirement in client.describe_use_case_requirements(use_case):
  pp.pprint(requirement.to_dict())

{ 'allowed_feature_mappings': { 'ACCOUNT_ID': { 'allowed_feature_types': [ 'CATEGORICAL'],
                                                'description': 'The unique '
                                                               'identifier for '
                                                               'each account.',
                                                'required': True},
                                'FRAUD_YN': { 'allowed_feature_types': [ 'CATEGORICAL'],
                                              'description': 'This specifies '
                                                             'whether the '
                                                             'account activity '
                                                             'was spurious or '
                                                             'not. Some '
                                                             'examples of '
                                                 

Finally, let's create the project.

In [ ]:
takeover_project = client.create_project(name='Insider Threat Detection', use_case=use_case)
takeover_project.to_dict()

{'created_at': '2021-11-22T21:16:20+00:00',
 'feature_groups_enabled': True,
 'name': 'Insider Threat Detection',
 'project_id': 'b89da65c6',
 'use_case': 'FRAUD_ACCOUNT'}

**Note: When feature_groups_enabled is True then the use case supports feature groups (collection of ML features). Feature groups are created at the organization level and can be tied to a project to further use it for training ML models**

## 2. Add Datasets to your Project

Abacus.AI can read datasets directly from `AWS S3` or `Google Cloud Storage` buckets, otherwise you can also directly upload and store your datasets with Abacus.AI. For this workshop, we will have Abacus.AI read the datasets directly from a public S3 bucket's location.

We are using one dataset for this notebook. We'll tell Abacus.AI how the dataset should be used when creating it by tagging the dataset with a special Abacus.AI **Dataset Type**.
- [Insider Threat Test Dataset](https://s3.amazonaws.com/realityengines.exampledatasets/fraud_transactions/creditcard.csv) (**ACTIVITY_LOG**): 
This dataset contains information about account takeovers that occurred for specified accounts & what class of takeover happened.

### Add the dataset to Abacus.AI

First we'll use Pandas to preview the file, then add it to Abacus.AI.

In [ ]:
pd.read_csv('https://s3.amazonaws.com/realityengines.exampledatasets/accounttakeover/kdd_500k.csv')

,account_id,duration,num_failed_logins,logged_in,num_file_creations,num_access_files,is_host_login,is_guest_login,class
0,4242436,0,0,0,0,0,0,0,smurf.
1,1882783,0,0,0,0,0,0,0,smurf.
2,2067606,0,0,0,0,0,0,0,smurf.
3,50774,0,0,1,0,0,0,0,normal.
4,719138,0,0,0,0,0,0,0,neptune.
...,...,...,...,...,...,...,...,...,...
545742,4867866,0,0,0,0,0,0,0,teardrop.
545743,4867867,0,0,0,0,0,0,0,teardrop.
545744,4867868,0,0,0,0,0,0,0,teardrop.
545745,4867869,0,0,0,0,0,0,0,teardrop.


Using the Create Dataset API, we can tell Abacus.AI the public S3 URI of where to find the datasets. We will also give each dataset a Refresh Schedule, which tells Abacus.AI when it should refresh the dataset (take an updated/latest copy of the dataset).

If you're unfamiliar with Cron Syntax, Crontab Guru can help translate the syntax back into natural language: [https://crontab.guru/#0_12_\*_\*_\*](https://crontab.guru/#0_12_*_*_*)

**Note: This cron string will be evaluated in UTC time zone**

In [ ]:
takeover_dataset = client.create_dataset_from_file_connector(name='Insider Threat Testing Data', 
                                                              table_name='Insider Threat Testing Data',
                                     location='s3://realityengines.exampledatasets/accounttakeover/kdd_500k.csv',
                                     refresh_schedule='0 12 * * *')
datasets = [takeover_dataset]

## 3. Create Feature Groups and add them to your Project

Datasets are created at the organization level and can be used to create feature groups as follows:

In [ ]:
feature_group = client.create_feature_group(table_name='test_account_takeover_and_defense',sql='SELECT * FROM Insider_Threat_Test_Data')

Adding Feature Group to the project:

In [ ]:
client.add_feature_group_to_project(feature_group_id=feature_group.feature_group_id,project_id = takeover_project.project_id)

Setting the Feature Group type according to the use case requirements:

In [ ]:
client.set_feature_group_type(feature_group_id=feature_group.feature_group_id, project_id = takeover_project.project_id, feature_group_type= "CUSTOM_TABLE")

Check current Feature Group schema:

In [ ]:
client.get_feature_group_schema(feature_group_id=feature_group.feature_group_id)

[Feature(name='account_id',
   select_clause=None,
   feature_mapping=None,
   source_table='Insider_Threat_Test_Data',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None), Feature(name='duration',
   select_clause=None,
   feature_mapping=None,
   source_table='Insider_Threat_Test_Data',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='NUMERICAL',
   data_type='INTEGER',
   columns=None,
   point_in_time_info=None), Feature(name='num_failed_logins',
   select_clause=None,
   feature_mapping=None,
   source_table='Insider_Threat_Test_Data',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='NUMERICAL',
   data_type='INTEGER',
   columns=None,
   point_in_time_info=None), Feature(name='logged_in',
   select_clause=None,
   feature_mapping=None

#### For each **Use Case**, there are special **Column Mappings** that must be applied to a column to fulfill use case requirements. We can find the list of available **Column Mappings** by calling the *Describe Use Case Requirements* API:

In [ ]:
client.describe_use_case_requirements(use_case)[0].allowed_feature_mappings

{'ACCOUNT_ID': {'allowed_feature_types': ['CATEGORICAL'],
  'description': 'The unique identifier for each account.',
  'required': True},
 'FRAUD_YN': {'allowed_feature_types': ['CATEGORICAL'],
  'description': 'This specifies whether the account activity was spurious or not. Some examples of spurious/fraudulent account activity are needed for our AI engine to create a predictive model.',
  'required': True},
 'IGNORE': {'description': 'Ignore this column in training',
  'multiple': True,
  'required': False}}

In [ ]:
client.set_feature_mapping(project_id = takeover_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='account_id', feature_mapping='ACCOUNT_ID')
client.set_feature_mapping(project_id = takeover_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='class', feature_mapping='FRAUD_YN')

[Feature(name='account_id',
   select_clause=None,
   feature_mapping='ACCOUNT_ID',
   source_table='Insider_Threat_Test_Data',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None), Feature(name='duration',
   select_clause=None,
   feature_mapping=None,
   source_table='Insider_Threat_Test_Data',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='NUMERICAL',
   data_type='INTEGER',
   columns=None,
   point_in_time_info=None), Feature(name='num_failed_logins',
   select_clause=None,
   feature_mapping=None,
   source_table='Insider_Threat_Test_Data',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='NUMERICAL',
   data_type='INTEGER',
   columns=None,
   point_in_time_info=None), Feature(name='logged_in',
   select_clause=None,
   feature_mapp

For each required Feature Group Type within the use case, you must assign the Feature group to be used for training the model:

In [ ]:
client.use_feature_group_for_training(project_id = takeover_project.project_id,feature_group_id= feature_group.feature_group_id)

Now that we've our feature groups assigned, we're almost ready to train a model!

To be sure that our project is ready to go, let's call project.validate to confirm that all the project requirements have been met:

In [ ]:
takeover_project.validate()

ProjectValidation(valid=True,
  dataset_errors=[],
  column_hints={})

## 4. Train a Model

For each **Use Case**, Abacus.AI has a bunch of options for training. We can call the *Get Training Config Options* API to see the available options.

In [ ]:
takeover_project.get_training_config_options()

[TrainingConfigOptions(name='TEST_SPLIT',
   data_type='INTEGER',
   value_type=None,
   value_options=None,
   value=None,
   default=10,
   options={'range': [5, 20]},
   description='Percent of dataset to use for test data. We support using a range between 5% to 20% of your dataset to use as test data.',
   required=None,
   last_model_value=None), TrainingConfigOptions(name='DROPOUT_RATE',
   data_type='INTEGER',
   value_type=None,
   value_options=None,
   value=None,
   default=None,
   options={'range': [0, 90]},
   description='Dropout percentage rate.',
   required=None,
   last_model_value=None), TrainingConfigOptions(name='BATCH_SIZE',
   data_type='ENUM',
   value_type=None,
   value_options=None,
   value=None,
   default=None,
   options={'values': [8, 16, 32, 64, 128, 256, 384, 512, 740, 1024]},
   description='Batch size.',
   required=None,
   last_model_value=None), TrainingConfigOptions(name='SAMPLING_UNIT_KEYS',
   data_type='ENUM_TAGS',
   value_type=None,
   valu

In this notebook, we'll just train with the default options, but definitely feel free to experiment, especially if you have familiarity with Machine Learning.

In [ ]:
takeover_model = takeover_project.train_model(training_config={})
takeover_model.to_dict()

{'created_at': '2021-11-22T21:24:33+00:00',
 'latest_model_version': {'dataset_versions': None,
  'error': None,
  'failed_deployment_ids': None,
  'model_config': {},
  'model_id': 'd00b77f48',
  'model_version': '167eba9534',
  'pending_deployment_ids': None,
  'status': 'PENDING',
  'training_completed_at': None,
  'training_started_at': None},
 'location': None,
 'model_config': {},
 'model_id': 'd00b77f48',
 'name': 'Insider Threat Detection Model',
 'predict_function_name': None,
 'project_id': 'b89da65c6',
 'refresh_schedules': None,
 'shared': False,
 'shared_at': None,
 'source_code': None,
 'train_function_name': None,
 'training_input_tables': None}

After we start training the model, we can call this blocking call that routinely checks the status of the model until it is trained and evaluated.

In [ ]:
takeover_model.wait_for_evaluation()

**Note that model training might take some minutes to some hours depending upon the size of datasets, complexity of the models being trained and a variety of other factors**

## **Checkpoint** [Optional]
As model training can take an hours to complete, your page could time out or you might end up hitting the refresh button, this section helps you restore your progress:

In [ ]:
!pip install abacusai
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)
api_key = ''  #@param {type: "string"}
from abacusai import ApiClient
client = ApiClient(api_key)
takeover_project = next(project for project in client.list_projects() if project.name == 'Insider Threat Detection')
takeover_model = takeover_project.list_models()[-1]
takeover_model.wait_for_evaluation()

## Evaluate your Model Metrics

After your model is done training you can inspect the model's quality by reviewing the model's metrics.

In [ ]:
pp.pprint(takeover_model.get_metrics().to_dict())

To get a better understanding on what these metrics mean, visit our [documentation](https://abacus.ai/app/help/useCases/FRAUD_TRANSACTIONS/training) page.

## 5. Deploy Model

After the model has been trained, we need to deploy the model to be able to start making predictions. Deploying a model will reserve cloud resources to host the model for Realtime and/or batch predictions.

In [ ]:
takeover_deployment=client.create_deployment(model_id=takeover_model.model_id,name='Insider Threat Deployment',description='Insider Threat Deployment', calls_per_second=3)
takeover_deployment.wait_for_deployment()

After the model is deployed, we need to create a deployment token for authenticating prediction requests. This token is only authorized to predict on deployments in this project, so it's safe to embed this token inside of a user-facing application or website.

In [ ]:
deployment_token = takeover_project.create_deployment_token().deployment_token
deployment_token

## 6. Predict


Now that you have an active deployment and a deployment token to authenticate requests, you can make the `predict_takeover` API call below.

This command will return the probability of a transaction being of each class of fraud. The prediction will be performed based on previously recorded frauds and the characteristics of the account that committed/didn't commit a fraud.

In [ ]:
ApiClient().predict_takeover(deployment_token=deployment_token, 
               deployment_id=takeover_deployment.deployment_id, 
               query_data={"duration":0,"num_failed_logins":1,"num_file_creations":0,"num_access_files":1,"logged_in":"0","is_host_login":"0","is_guest_login":"0"})